In [ ]:
import os
import requests
import feedparser
import pytz
import google.generativeai as genai
from datetime import datetime, timedelta

# ─── 환경변수 (GitHub Actions Secrets에서 가져옴) ─────────────────────
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
GEMINI_API_KEY      = os.getenv('GOOGLE_API_KEY')
CITY                = os.getenv("CITY_NAME", "Seoul,KR") # 기본값 설정
DISCORD_WEBHOOK     = os.getenv("DISCORD_WEBHOOK_URL")

# ─── 고정값 설정 ──────────────────────────────────────────────────
NEWS_RSS_URLS = [
    "https://feeds.bbci.co.uk/news/business/rss.xml",
    "https://feeds.bbci.co.uk/news/science_and_environment/rss.xml",
    "https://feeds.bbci.co.uk/news/technology/rss.xml"
]
GAMING_RSS_URLS = [
    "https://webzine.inven.co.kr/news/rss.php",
    "https://www.gamedeveloper.com/rss.xml",
    "https://game.donga.com/feeds/rss/",
    "https://www.gametoc.co.kr/rss/S1N86.xml",
    "https://bbs.ruliweb.com/news/537/rss"
]
TZ = pytz.timezone("Asia/Seoul")
# ──────────────────────────────────────────────────────────────────

# 💡 API 키 유효성 검사 및 모델 초기화
if not GEMINI_API_KEY:
    raise ValueError("GOOGLE_API_KEY secret is not set.")
if not OPENWEATHER_API_KEY:
    raise ValueError("OPENWEATHER_API_KEY secret is not set.")
if not DISCORD_WEBHOOK:
    raise ValueError("DISCORD_WEBHOOK_URL secret is not set.")

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# ----------------------------------------------------------------------
# HELPER FUNCTIONS
# ----------------------------------------------------------------------

# 1) 날씨 관련 함수
def fetch_weather():
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": CITY, "appid": OPENWEATHER_API_KEY, "units": "metric", "lang": "kr"}
    r = requests.get(url, params=params); r.raise_for_status()
    current = r.json()

    forecast_url = "https://api.openweathermap.org/data/2.5/forecast"
    r = requests.get(forecast_url, params=params); r.raise_for_status()
    forecast = r.json()

    hourly_temps = []
    now = datetime.now(TZ)
    for item in forecast['list']:
        dt = datetime.fromtimestamp(item['dt'], TZ)
        if dt <= now + timedelta(hours=24):
            hourly_temps.append({
                "time": dt.strftime("%H:%M"),
                "temp": item['main']['temp'],
                "icon": item['weather'][0]['icon']
            })
    return {
        "current": {
            "desc": current["weather"][0]["description"].capitalize(),
            "temp": current["main"]["temp"],
            "feels": current["main"]["feels_like"],
            "humidity": current["main"]["humidity"],
            "icon": current["weather"][0]["icon"],
        },
        "hourly": hourly_temps
    }

def create_temperature_graph(hourly_temps):
    graph_width = min(len(hourly_temps), 8)
    step = max(1, len(hourly_temps) // graph_width)
    points = hourly_temps[::step][:graph_width]

    temps = [pt['temp'] for pt in points]
    min_temp, max_temp = min(temps), max(temps)
    temp_range = max_temp - min_temp or 1
    max_bar = 20

    lines = []
    for pt in points:
        length = int((pt['temp'] - min_temp) / temp_range * max_bar)
        bars = '█' * length
        lines.append(f"{pt['time']:>5} | {bars:<{max_bar}} {pt['temp']:.1f}°C")
    return "\n".join(lines)

def build_weather_embed(data):
    icon_url = f"https://openweathermap.org/img/wn/{data['current']['icon']}@2x.png"
    title = f"🏙️ {CITY} 오늘의 날씨 ({datetime.now(TZ).strftime('%Y-%m-%d')})"
    graph = create_temperature_graph(data['hourly'])
    hourly_text = f"```\n{graph}\n```"
    return {
        "title": title, "description": data["current"]["desc"], "color": 0x3498db,
        "thumbnail": {"url": icon_url},
        "fields": [
            {"name": "🌡️ 현재 온도", "value": f"{data['current']['temp']}°C", "inline": True},
            {"name": "🤗 체감 온도", "value": f"{data['current']['feels']}°C", "inline": True},
            {"name": "💧 습도", "value": f"{data['current']['humidity']}%", "inline": True},
            {"name": "📊 시간별 기온 그래프", "value": hourly_text, "inline": False},
        ], "footer": {"text": "Powered by OpenWeatherMap"},
    }

# 2) 뉴스 관련 함수
def fetch_recent_entries(rss_urls):
    now = datetime.now(TZ)
    start = now - timedelta(hours=24)
    entries = []
    for rss_url in rss_urls:
        try:
            feed = feedparser.parse(rss_url)
            category = feed.feed.title if hasattr(feed.feed, 'title') else rss_url.split('/')[-2].replace('_', ' ').title()
            for e in feed.entries:
                try:
                    pub_time_struct = e.get('published_parsed') or e.get('updated_parsed')
                    if pub_time_struct:
                        pub = datetime(*pub_time_struct[:6], tzinfo=pytz.utc).astimezone(TZ)
                        if pub >= start and hasattr(e, 'title') and hasattr(e, 'link'):
                            entries.append(f"- [{category}] {e.title.strip()} ({e.link.strip()})")
                except Exception as entry_error:
                    continue
        except Exception as feed_error:
            print(f"Error fetching RSS feed {rss_url}: {feed_error}")
            continue
    return entries

def summarize_news_with_gemini(entries, prompt_template):
    if not entries:
        return "최근 24시간 이내 새로운 뉴스가 없습니다."
    prompt = prompt_template + "\n".join(entries)
    try:
        res = model.generate_content(prompt)
        return res.text
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return "뉴스 요약 생성 중 오류가 발생했습니다."

def build_news_embed(summary):
    return {
        "title": f"📰 세계 뉴스 요약 ({(datetime.now(TZ)).strftime('%Y-%m-%d')})",
        "description": summary, "color": 0x2ecc71,
        "footer": {"text": "Powered by Google Gemini & BBC RSS"},
    }

def build_gaming_news_embed(summary):
    return {
        "title": f"🎮 게임 뉴스 요약 ({(datetime.now(TZ)).strftime('%Y-%m-%d')})",
        "description": summary, "color": 0x9b59b6,
        "footer": {"text": "Powered by Google Gemini & Various Gaming RSS"},
    }

def build_gaming_trends_embed(analysis):
    return {
        "title": f"📊 게임 트렌드 분석 ({(datetime.now(TZ)).strftime('%Y-%m-%d')})",
        "description": analysis, "color": 0x3498db,
        "footer": {"text": "Powered by Google Gemini & Various Gaming RSS"},
    }

# 3) 디스코드 전송 함수
def send_to_discord(embeds):
    for embed in embeds:
        payload = {"embeds": [embed]}
        try:
            r = requests.post(DISCORD_WEBHOOK, json=payload)
            r.raise_for_status()
        except Exception as e:
            print(f"Error sending embed to Discord: {e}")
            continue

# ----------------------------------------------------------------------
# MAIN EXECUTION
# ----------------------------------------------------------------------

def run_daily_briefing():
    print("✅ Daily briefing script started...")
    all_embeds = []

    # 날씨
    print("🌦️ Fetching weather...")
    try:
        wdata = fetch_weather()
        wembed = build_weather_embed(wdata)
        all_embeds.append(wembed)
        print("👍 Weather info processed!")
    except Exception as e:
        print(f"❌ Error in weather processing: {e}")

    # 일반 뉴스
    print("📰 Fetching general news...")
    try:
        entries = fetch_recent_entries(NEWS_RSS_URLS)
        news_prompt = "아래 뉴스 목록을 보고, 중요한 이슈 3~5개를 중요도 순으로 요약해줘. 각 항목은 '[분야] 제목' 형식으로 시작하고, 핵심 내용과 원문 링크를 포함해줘.\n\n뉴스 목록:\n"
        summary = summarize_news_with_gemini(entries, news_prompt)
        nembed = build_news_embed(summary)
        all_embeds.append(nembed)
        print("👍 General news processed!")
    except Exception as e:
        print(f"❌ Error in general news processing: {e}")

    # 게임 뉴스 및 트렌드
    print("🎮 Fetching gaming news...")
    try:
        gaming_entries = fetch_recent_entries(GAMING_RSS_URLS)
        if gaming_entries:
            gaming_summary_prompt = "아래 게임 뉴스 목록을 보고, 흥미로운 주요 소식 3~5개를 요약해줘. 각 항목은 '[출처] 제목'으로 시작하고, 핵심 내용과 원문 링크를 포함해줘.\n\n뉴스 목록:\n"
            gaming_summary = summarize_news_with_gemini(gaming_entries, gaming_summary_prompt)
            gembed = build_gaming_news_embed(gaming_summary)
            all_embeds.append(gembed)

            trends_prompt = "아래 게임 뉴스 목록을 분석하여 주요 트렌드, 핵심 키워드, 주목할 만한 게임/이벤트를 정리해줘.\n\n뉴스 목록:\n"
            trends_analysis = summarize_news_with_gemini(gaming_entries, trends_prompt)
            tembed = build_gaming_trends_embed(trends_analysis)
            all_embeds.append(tembed)
            print("👍 Gaming news processed!")
        else:
            print("ℹ️ No new gaming news.")
    except Exception as e:
        print(f"❌ Error in gaming news processing: {e}")

    # 수집된 모든 임베드 전송
    if all_embeds:
        print(f"🚀 Sending {len(all_embeds)} embeds to Discord...")
        send_to_discord(all_embeds)
        print("🎉 All tasks completed!")
    else:
        print("🤔 No content to send.")

# 💥 스크립트 실행
if __name__ == "__main__":
    run_daily_briefing()